In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, roc_auc_score, mean_squared_error, accuracy_score, mean_absolute_error, precision_score, recall_score, f1_score
from pickle import dump
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Concatenate
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

pd.set_option('display.max_columns', 200)

C:\Users\admin\anaconda3\envs\tensorflow-cuda\lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
hospital1 = 'Anam_hospital'
hospital2 = 'Guro_hospital'
hospital3 = 'Ansan_hospital'

# Model train

In [4]:
def nan_del(X, y):
    na_index = np.where(np.isnan(y))[0]
    del_na_X = np.delete(X, na_index,axis=0)
    del_na_y = np.delete(y, na_index)
    
    return del_na_X, del_na_y

In [5]:
def performacne_model(Real_Test_y, y_pred, y_pred_prob):
    accuracy= accuracy_score(Real_Test_y, y_pred)
    precision = precision_score(Real_Test_y, y_pred)
    recall = recall_score(Real_Test_y, y_pred)
    f1 = f1_score(Real_Test_y, y_pred)
    auc = roc_auc_score(Real_Test_y, y_pred_prob)
    prc = average_precision_score(Real_Test_y, y_pred_prob)
    confusion = confusion_matrix(Real_Test_y, y_pred)
    
    return [accuracy, precision, recall, f1, auc, prc, confusion]

In [7]:
# Standard Deep Learning code
class MLP_Layer3():
    def base_node1(self, x, node1, drop_out_rate1):
        x = Dense(node1)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out_rate1)(x)
        return x
    
    def base_node2(self, x, node2, drop_out_rate2):
        x = Dense(node2)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out_rate2)(x)
        return x

    def build_branch(self, x, node1, drop_out_rate1, node2, drop_out_rate2, node3, drop_out_rate3):   
        x= self.base_node1(x, node1, drop_out_rate1)
        x= self.base_node2(x, node2, drop_out_rate2)
        x = Dense(node3)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out_rate3)(x)
        x = Dense(1)(x)
        x = Activation("sigmoid", name="output")(x)
        return x
    
    def assemble_full_model(self, feature, node1=64, drop_out_rate1=0.1, node2=64, drop_out_rate2=0.1,node3=64, drop_out_rate3=0.1):
        input_shape = (feature,)
        inputs = Input(shape=input_shape)
        branch = self.build_branch(inputs, node1, drop_out_rate1, node2, drop_out_rate2, node3, drop_out_rate3)
        model = Model(inputs=inputs,
                     outputs = branch,
                     name="anti_net")
        return model
    
def model2_builder(node1=64, drop_out_rate1=0.1, node2=64, drop_out_rate2=0.1, node3=64, drop_out_rate3=0.1):
    loss_dict ={'output': 'binary_crossentropy'}
    metrics_dict = {'output': 'accuracy'}

    model = MLP_Layer3().assemble_full_model(feature_size, node1, drop_out_rate1, node2, drop_out_rate2, node3, drop_out_rate3)

    model.compile(optimizer='adam',
                  loss=loss_dict,
                  metrics=metrics_dict)

    return model

In [9]:
def performacne_model(Real_Test_y, y_pred, y_pred_prob):
    accuracy= accuracy_score(Real_Test_y, y_pred)
    precision = precision_score(Real_Test_y, y_pred)
    recall = recall_score(Real_Test_y, y_pred)
    f1 = f1_score(Real_Test_y, y_pred)
    auc = roc_auc_score(Real_Test_y, y_pred_prob)
    prc = average_precision_score(Real_Test_y, y_pred_prob)
    confusion = confusion_matrix(Real_Test_y, y_pred)
    
    return [accuracy, precision, recall, f1, auc, prc, confusion]

In [11]:
def MLP2_tuning_predict(Train_X_sc_MI, Train_y, Real_Test_X_sc, Real_Test_y, External_Test_X1, External_Test_y1):
    node1 = [64, 256]
    node2 = [64, 256]
    node3 = [64, 256]
    drop_out_rate1 = [0.1,0.3]
    drop_out_rate2 = [0.1,0.3]
    drop_out_rate3 = [0.1,0.3]
   
    param_grid2 = {'node1': node1,'node2': node2,'node3': node3,'drop_out_rate1': drop_out_rate1,'drop_out_rate2': drop_out_rate2,'drop_out_rate3': drop_out_rate3}

    model = KerasClassifier(build_fn=model2_builder, verbose=0)

    grid = GridSearchCV(estimator=model, param_grid=param_grid2, cv=5)
    gscv = grid.fit(Train_X_sc_MI, Train_y)

    y_pred = gscv.predict(Real_Test_X_sc)
    y_pred_prob = gscv.predict_proba(Real_Test_X_sc)[:, 1]
    performance = performacne_model(Real_Test_y, y_pred, y_pred_prob)

    external_y_pred1 = gscv.predict(External_Test_X1)
    external_y_pred_prob1 = gscv.predict_proba(External_Test_X1)[:, 1]
    external_performance1 = performacne_model(External_Test_y1, external_y_pred1, external_y_pred_prob1)
    
    return performance, external_performance1, gscv

In [14]:
count=0
external_count=0
target_name = ['tgc','m','fgc','f','cp','c','a','ag','g']
perform = ['accuracy','precision','recall','f1','auc','prc']
target_list =['train_length','valid_length','test_length']
for t in target_name:
    for p in perform:
        target_list.append(f'{t}_{p}')
performance_result_df = pd.DataFrame(columns=target_list)
performance_external_result_df1 = pd.DataFrame(columns=target_list)

for seed in range(count, 30):
    print(seed)
    Train_X_MI = np.load('Preprocessed training data X in .npy format')
    Train_y = np.load('Preprocessed training data y in .npy format')
    Test_X_MI= np.load('Preprocessed internal test data X in .npy format')
    Test_y = np.load('Preprocessed internal test data y in .npy format')
    external_test_X1 = np.load('Preprocessed external test data X in .npy format')
    external_test_y1 = np.load('Preprocessed external test data y in .npy format')
    
    tgc_train_y = Train_y[:,0]
    m_train_y = Train_y[:,1]
    fgc_train_y = Train_y[:,2]
    f_train_y = Train_y[:,3]
    cp_train_y = Train_y[:,4]
    c_train_y = Train_y[:,5]
    a_train_y = Train_y[:,6]
    ag_train_y = Train_y[:,7]
    g_train_y = Train_y[:,8]

    tgc_test_y = Test_y[:,0]
    m_test_y = Test_y[:,1]
    fgc_test_y = Test_y[:,2]
    f_test_y = Test_y[:,3]
    cp_test_y = Test_y[:,4]
    c_test_y = Test_y[:,5]
    a_test_y = Test_y[:,6]
    ag_test_y = Test_y[:,7]
    g_test_y = Test_y[:,8]

    external_tgc_test_y1 = external_test_y1[:,0]
    external_m_test_y1 = external_test_y1[:,1]
    external_fgc_test_y1 = external_test_y1[:,2]
    external_f_test_y1 = external_test_y1[:,3]
    external_cp_test_y1 = external_test_y1[:,4]
    external_c_test_y1 = external_test_y1[:,5]
    external_a_test_y1 = external_test_y1[:,6]
    external_ag_test_y1 = external_test_y1[:,7]
    external_g_test_y1 = external_test_y1[:,8]

    train_y_list = [tgc_train_y,m_train_y,fgc_train_y,f_train_y,cp_train_y,c_train_y,a_train_y,ag_train_y,g_train_y]
    test_y_list = [tgc_test_y,m_test_y,fgc_test_y,f_test_y,cp_test_y,c_test_y,a_test_y,ag_test_y,g_test_y]
    external_test_y1_list = [external_tgc_test_y1,external_m_test_y1,external_fgc_test_y1,external_f_test_y1,external_cp_test_y1,external_c_test_y1,external_a_test_y1,external_ag_test_y1,external_g_test_y1]

    Train_index, Val_index = train_test_split(range(len(Train_X_MI)), test_size=0.3, random_state=seed)
    X_val = Train_X_MI[Val_index]
    Y_val_tgc = train_y_list[0][Val_index]
    Y_val_m = train_y_list[1][Val_index]
    Y_val_fgc = train_y_list[2][Val_index]
    Y_val_f = train_y_list[3][Val_index]
    Y_val_cp = train_y_list[4][Val_index]
    Y_val_c = train_y_list[5][Val_index]
    Y_val_a = train_y_list[6][Val_index]
    Y_val_ag = train_y_list[7][Val_index]
    Y_val_g = train_y_list[8][Val_index]


    X_train = Train_X_MI[Train_index]
    Y_train_tgc = train_y_list[0][Train_index]
    Y_train_m = train_y_list[1][Train_index]
    Y_train_fgc = train_y_list[2][Train_index]
    Y_train_f = train_y_list[3][Train_index]
    Y_train_cp = train_y_list[4][Train_index]
    Y_train_c = train_y_list[5][Train_index]
    Y_train_a = train_y_list[6][Train_index]
    Y_train_ag = train_y_list[7][Train_index]
    Y_train_g = train_y_list[8][Train_index]

    X_test = Test_X_MI
    Y_test_tgc = test_y_list[0]
    Y_test_m = test_y_list[1]
    Y_test_fgc = test_y_list[2]
    Y_test_f = test_y_list[3]
    Y_test_cp = test_y_list[4]
    Y_test_c = test_y_list[5]
    Y_test_a = test_y_list[6]
    Y_test_ag = test_y_list[7]
    Y_test_g = test_y_list[8]

    Y_test_tgc = test_y_list[0]
    Y_test_m = test_y_list[1]
    Y_test_fgc = test_y_list[2]
    Y_test_f = test_y_list[3]
    Y_test_cp = test_y_list[4]
    Y_test_c = test_y_list[5]
    Y_test_a = test_y_list[6]
    Y_test_ag = test_y_list[7]
    Y_test_g = test_y_list[8]

    external_Y_test1_tgc = external_test_y1_list[0]
    external_Y_test1_m = external_test_y1_list[1]
    external_Y_test1_fgc = external_test_y1_list[2]
    external_Y_test1_f = external_test_y1_list[3]
    external_Y_test1_cp = external_test_y1_list[4]
    external_Y_test1_c = external_test_y1_list[5]
    external_Y_test1_a = external_test_y1_list[6]
    external_Y_test1_ag = external_test_y1_list[7]
    external_Y_test1_g = external_test_y1_list[8]

    feature_size = X_train.shape[1]

    Y_train =[Y_train_tgc,Y_train_m,Y_train_fgc,Y_train_f,Y_train_cp,Y_train_c,Y_train_a,Y_train_ag,Y_train_g]
    Y_val =[Y_val_tgc,Y_val_m,Y_val_fgc,Y_val_f,Y_val_cp,Y_val_c,Y_val_a,Y_val_ag,Y_val_g]
    Y_test =[Y_test_tgc,Y_test_m,Y_test_fgc,Y_test_f,Y_test_cp,Y_test_c,Y_test_a,Y_test_ag,Y_test_g]
    external_Y_test1 =[external_Y_test1_tgc,external_Y_test1_m,external_Y_test1_fgc,external_Y_test1_f,external_Y_test1_cp,external_Y_test1_c,external_Y_test1_a,external_Y_test1_ag,external_Y_test1_g]

    performance_result_df.loc[count,'train_length'] = len(X_train)
    performance_result_df.loc[count,'valid_length'] = len(X_val)
    performance_result_df.loc[count,'test_length'] = len(X_test)

    tmp_train_X = []
    tmp_val_X = []
    tmp_test_X = []
    tmp_external_X1 = []
    tmp_train_y = []
    tmp_val_y = []
    tmp_test_y = []
    tmp_external_y1 = []

    for i,(train, val, test, external_test1) in enumerate(zip(Y_train, Y_val, Y_test, external_Y_test1)):
        tmp_X, tmp_y = nan_del(X_train, train)
        tmp_train_X.append(tmp_X)
        tmp_train_y.append(tmp_y)

        tmp_X, tmp_y = nan_del(X_val, val)
        tmp_val_X.append(tmp_X)
        tmp_val_y.append(tmp_y)

        tmp_X, tmp_y = nan_del(X_test, test)
        tmp_test_X.append(tmp_X)
        tmp_test_y.append(tmp_y)

        tmp_X, tmp_y = nan_del(external_test_X1, external_test1)
        tmp_external_X1.append(tmp_X)
        tmp_external_y1.append(tmp_y)
        

    for i in range(len(target_name)):
        tmp_val_train_X = np.concatenate([tmp_train_X[i], tmp_val_X[i]],axis=0)
        tmp_val_train_y = np.concatenate([tmp_train_y[i], tmp_val_y[i]])
        
        # Layer 3
        internal_performance, external_performance1, model_info = MLP2_tuning_predict(tmp_val_train_X, tmp_val_train_y, tmp_test_X[i], tmp_test_y[i], tmp_external_X1[i], tmp_external_y1[i])
        filename = 'Model save path in .h5 format'
        model_info.best_estimator_.model.save(filename)
        for j, performance_name in enumerate(perform):
            target_feature = target_name[i]+'_'+performance_name
            performance_result_df.loc[count, target_feature] = internal_performance[j]
            performance_external_result_df1.loc[count, target_feature] = external_performance1[j]
        del model_info
        
    count+=1

27


C:\Users\admin\AppData\Local\Temp\ipykernel_50892\247538212.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=model2_builder, verbose=0)


293/293 [==============================] - 0s 1ms/step


C:\Users\admin\AppData\Local\Temp\ipykernel_50892\247538212.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=model2_builder, verbose=0)


120/120 [==============================] - 0s 846us/step


...

72/72 [==============================] - 0s 1ms/step


C:\Users\admin\AppData\Local\Temp\ipykernel_50892\247538212.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=model2_builder, verbose=0)


360/360 [==============================] - 0s 1ms/step


C:\Users\admin\AppData\Local\Temp\ipykernel_50892\247538212.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=model2_builder, verbose=0)


177/177 [==============================] - 0s 953us/step


In [16]:
performance_result_df

,train_length,valid_length,test_length,tgc_accuracy,tgc_precision,tgc_recall,tgc_f1,tgc_auc,tgc_prc,m_accuracy,m_precision,m_recall,m_f1,m_auc,m_prc,fgc_accuracy,fgc_precision,fgc_recall,fgc_f1,fgc_auc,fgc_prc,f_accuracy,f_precision,f_recall,f_f1,f_auc,f_prc,cp_accuracy,cp_precision,cp_recall,cp_f1,cp_auc,cp_prc,c_accuracy,c_precision,c_recall,c_f1,c_auc,c_prc,a_accuracy,a_precision,a_recall,a_f1,a_auc,a_prc,ag_accuracy,ag_precision,ag_recall,ag_f1,ag_auc,ag_prc,g_accuracy,g_precision,g_recall,g_f1,g_auc,g_prc
0,74769.0,32044.0,45734.0,0.725230,0.765263,0.837800,0.799890,0.787206,0.871580,0.735114,0.741921,0.969663,0.840640,0.759956,0.884111,0.735437,0.738109,0.651374,0.692035,0.785744,0.754571,0.704631,0.732449,0.830941,0.778593,0.762033,0.828507,0.729167,0.791395,0.749039,0.769635,0.791235,0.851700,0.753475,0.736937,0.641677,0.686016,0.803421,0.744071,0.757683,0.694795,0.338087,0.454846,0.688312,0.526606,0.679072,0.728596,0.484586,0.582052,0.706944,0.676271,0.864314,0.630664,0.348420,0.448860,0.764959,0.449019
1,74602.0,31973.0,45972.0,0.727444,0.780433,0.815493,0.797578,0.790523,0.877775,0.733549,0.752680,0.941815,0.836692,0.763202,0.893957,0.736775,0.745318,0.645437,0.691791,0.784149,0.758504,0.715456,0.761656,0.795455,0.778189,0.773543,0.846373,0.727012,0.784068,0.756943,0.770267,0.788682,0.848994,0.754852,0.760970,0.612958,0.678991,0.803138,0.752631,0.754202,0.680672,0.318741,0.434171,0.683314,0.511012,0.678509,0.698350,0.538551,0.608128,0.707125,0.670287,0.861431,0.686105,0.242001,0.357800,0.773070,0.464832
2,74182.0,31793.0,46572.0,0.735526,0.792204,0.821153,0.806419,0.793667,0.884395,0.736284,0.745905,0.963600,0.840891,0.762210,0.889334,0.733491,0.749466,0.648023,0.695063,0.786833,0.763719,0.718724,0.758949,0.815652,0.786280,0.772986,0.849616,0.733344,0.790064,0.774229,0.782066,0.795943,0.856260,0.750080,0.761897,0.618267,0.682609,0.802266,0.757981,0.757534,0.667937,0.343291,0.453502,0.694978,0.512855,0.678014,0.702219,0.530027,0.604092,0.701508,0.671614,0.866268,0.641493,0.375652,0.473833,0.768326,0.449910
3,74783.0,32051.0,45713.0,0.736316,0.779788,0.839793,0.808679,0.793986,0.878222,0.729117,0.761949,0.908655,0.828861,0.754729,0.886837,0.730453,0.748242,0.614796,0.674987,0.778835,0.743494,0.713253,0.753678,0.806801,0.779335,0.767529,0.838672,0.729871,0.793132,0.751878,0.771954,0.792918,0.852970,0.752537,0.759684,0.594628,0.667098,0.799938,0.743516,0.761964,0.680719,0.260858,0.377178,0.684034,0.497936,0.680007,0.714359,0.503399,0.590606,0.705549,0.668804,0.869723,0.716931,0.279094,0.401779,0.772691,0.486454
4,74405.0,31888.0,46254.0,0.730724,0.777509,0.833783,0.804663,0.789688,0.879033,0.734134,0.758179,0.930660,0.835612,0.757053,0.889833,0.731968,0.762143,0.610714,0.678077,0.780341,0.756185,0.716564,0.763850,0.798952,0.781007,0.770743,0.842904,0.727028,0.784729,0.764444,0.774454,0.790862,0.855731,0.753478,0.759963,0.619776,0.682747,0.807151,0.762912,0.755352,0.685714,0.305549,0.422732,0.696623,0.523517,0.680694,0.715769,0.523094,0.604449,0.708726,0.680491,0.859246,0.665750,0.290408,0.404408,0.768229,0.461252
5,74812.0,32063.0,45672.0,0.734475,0.790363,0.818075,0.803980,0.794105,0.881808,0.736081,0.763798,0.917492,0.833620,0.760797,0.880527,0.737281,0.771355,0.617980,0.686202,0.786316,0.754975,0.711377,0.782607,0.751321,0.766645,0.771600,0.844732,0.730842,0.800727,0.743499,0.771053,0.794125,0.853367,0.751550,0.755165,0.607716,0.673465,0.802715,0.748137,0.750166,0.663352,0.242346,0.354998,0.682911,0.494781,0.682485,0.715461,0.525923,0.606222,0.707620,0.675374,0.863789,0.658698,0.324480,0.434783,0.765953,0.466862
6,74771.0,32046.0,45730.0,0.728177,0.784866,0.811771,0.798092,0.791264,0.879917,0.733608,0.752681,0.935846,0.834329,0.759987,0.884667,0.736918,0.763825,0.627125,0.688758,0.786646,0.757760,0.686135,0.726675,0.799058,0.761150,0.754433,0.831951,0.729964,0.806740,0.731746,0.767415,0.793578,0.854797,0.753113,0.765637,0.609966,0.678994,0.804189,0.758777,0.750782,0.683450,0.329251,0.444408,0.679258,0.520388,0.681899

In [17]:
performance_external_result_df1

,train_length,valid_length,test_length,tgc_accuracy,tgc_precision,tgc_recall,tgc_f1,tgc_auc,tgc_prc,m_accuracy,m_precision,m_recall,m_f1,m_auc,m_prc,fgc_accuracy,fgc_precision,fgc_recall,fgc_f1,fgc_auc,fgc_prc,f_accuracy,f_precision,f_recall,f_f1,f_auc,f_prc,cp_accuracy,cp_precision,cp_recall,cp_f1,cp_auc,cp_prc,c_accuracy,c_precision,c_recall,c_f1,c_auc,c_prc,a_accuracy,a_precision,a_recall,a_f1,a_auc,a_prc,ag_accuracy,ag_precision,ag_recall,ag_f1,ag_auc,ag_prc,g_accuracy,g_precision,g_recall,g_f1,g_auc,g_prc
0,NaN,NaN,NaN,0.719821,0.746960,0.871706,0.804526,0.785253,0.874994,0.750323,0.754468,0.977146,0.851489,0.786185,0.908954,0.719914,0.737819,0.598875,0.661126,0.760872,0.728792,0.695417,0.714103,0.801758,0.755396,0.768299,0.821994,0.725855,0.801653,0.735198,0.766989,0.789669,0.853608,0.757221,0.733034,0.636919,0.681605,0.800321,0.738993,0.838238,0.668394,0.192633,0.299073,0.634375,0.375421,0.727695,0.719218,0.487455,0.581079,0.723755,0.641046,0.893874,0.715818,0.408361,0.520045,0.789589,0.498379
1,NaN,NaN,NaN,0.724080,0.788511,0.796471,0.792471,0.784656,0.874147,0.749389,0.775306,0.926336,0.844119,0.779028,0.906598,0.723114,0.751704,0.586962,0.659196,0.761648,0.734090,0.703483,0.738206,0.766262,0.751972,0.771118,0.826157,0.722802,0.779945,0.763826,0.771801,0.789207,0.854373,0.758953,0.765709,0.589610,0.666219,0.799139,0.742328,0.842964,0.680758,0.232454,0.346568,0.666707,0.387560,0.723089,0.688836,0.520290,0.592816,0.725792,0.633825,0.885655,0.725382,0.302320,0.426772,0.800713,0.514411
2,NaN,NaN,NaN,0.721293,0.777304,0.810977,0.793784,0.782018,0.873437,0.745438,0.757690,0.959245,0.846637,0.780428,0.908242,0.719859,0.753778,0.573174,0.651185,0.761616,0.733414,0.698615,0.725422,0.782341,0.752807,0.768515,0.823853,0.725409,0.780917,0.768036,0.774423,0.791711,0.855099,0.754196,0.766240,0.572067,0.655067,0.792440,0.738661,0.837346,0.633478,0.218517,0.324944,0.652311,0.356423,0.726998,0.703709,0.510145,0.591494,0.716615,0.636417,0.896099,0.707258,0.447107,0.547868,0.790840,0.504006
3,NaN,NaN,NaN,0.722818,0.765542,0.837407,0.799863,0.784979,0.874011,0.750503,0.781760,0.914762,0.843047,0.785348,0.909763,0.722562,0.754797,0.580431,0.656229,0.760331,0.728030,0.700789,0.728053,0.782035,0.754079,0.770336,0.826211,0.724610,0.781218,0.765701,0.773382,0.791062,0.856318,0.756644,0.765666,0.581518,0.661006,0.795701,0.737423,0.832085,0.649289,0.136386,0.225422,0.683122,0.394668,0.728055,0.714532,0.496400,0.585820,0.729389,0.640742,0.888131,0.735673,0.320673,0.446654,0.801806,0.520040
4,NaN,NaN,NaN,0.715694,0.743930,0.869441,0.801803,0.784762,0.874632,0.749389,0.782186,0.911770,0.842022,0.780463,0.904265,0.721073,0.760120,0.567791,0.650028,0.758775,0.734090,0.701795,0.731518,0.776706,0.753435,0.769457,0.822951,0.724516,0.802140,0.731563,0.765227,0.791149,0.857940,0.758883,0.761581,0.595439,0.668339,0.795673,0.741317,0.830212,0.623529,0.131906,0.217749,0.668035,0.368827,0.727801,0.710362,0.502182,0.588401,0.728742,0.641627,0.885619,0.718268,0.308692,0.431806,0.784160,0.489967
5,NaN,NaN,NaN,0.719637,0.759172,0.843806,0.799255,0.782554,0.871854,0.747988,0.796629,0.880824,0.836613,0.786007,0.907007,0.721845,0.773660,0.551705,0.644098,0.758697,0.726788,0.705783,0.761093,0.726479,0.743384,0.770469,0.824277,0.723271,0.798064,0.735084,0.765280,0.788811,0.853276,0.759553,0.761062,0.598608,0.670130,0.797157,0.736921,0.832174,0.648712,0.137880,0.227422,0.690662,0.384192,0.728160,0.714040,0.497655,0.586526,0.729046,0.642184,0.893156,0.737688,0.374203,0.496533,0.798591,0.513350
6,NaN,NaN,NaN,0.722766,0.774131,0.820158,0.796480,0.783967,0.874857,0.749749,0.774722,0.928298,0.844585,0.784140,0.909781,0.723445,0.768603,0.563437,0.650220,0.760087,0.730832,0.670517,0.695220,0.780473,0.735384,0.753503,0.814659,0.724399,0.811945,0.716981,0.761514,0.787588,0.854616,0.757937,0.762779,0.590289,0.665539,0.796941,0.742300,0.839576,0.670455,0.205575,0.314667,0.670594,0.384964,0.726047,0.706570,0.500927,0.586238,0.724761,0.635758,0.884722,0.701416,0.315575,0.435302,0.

In [21]:
performance_result_df.to_csv('model internal test performance file save path')
performance_external_result_df1.to_csv('model external test performance file save path')